[Project Solution 1](https://www.kaggle.com/code/rodrigoleyva2/google-data-analytics-cyclistic-capstone-project/notebook)

[Project Solution 2](https://www.kaggle.com/code/chingheichau/google-data-analytics-capstone-project-1/notebook)

[Project Solution 3](https://www.kaggle.com/code/jhelisonuchoa/google-data-analytics-capstone-case-study-1) - Required Solution